# **<font color='black'>COLOMBIA</font>**

In [50]:
import logging
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("pyspark").setLevel(logging.ERROR)
import warnings
warnings.filterwarnings('ignore')
# Paquetes (checar o que preciso e o que não preciso)
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


#logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
import shutil

from datetime import date, datetime, timedelta, date
import pandas as pd
from pandas import Series
import numpy as np
from math import sqrt
import scipy.stats as sps
import seaborn as sns
import matplotlib as m
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.pyplot import figure
from matplotlib.pylab import rcParams
from datetime import datetime
from dateutil.relativedelta import relativedelta

import requests
from statsforecast.arima import AutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
import sklearn
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from prophet import Prophet

import os
import pyspark.sql.functions as F
import dotenv
from gcpspark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, date_sub, col

In [51]:
dataset_id = 'data_analytics_default'
table_id = 'BQ_CO_SM_SALES_02'
ENV = os.environ["ENVIRONMENT"]

In [3]:
cota_recencia = 3
cota_frecuencia = 180
cota_latencia = 2
cota_mediana = 0

In [4]:
ENSEMBLE = 2
COTA_MAPE_MODELS = 24

In [5]:
# Get the current date
current_date = datetime.now()
# Get the day of the week (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
day_of_week = current_date.weekday()
# Set MONDAY variable
MONDAY = day_of_week == 0 or True

In [6]:
# Fechas de ejecución, entreinamiento y inicio y historial de 13 meses
fecha_ejecucion = date.today() - timedelta(days = datetime.weekday(date.today())) - timedelta(days = 0)
fecha_train = fecha_ejecucion - timedelta(days = 8)

mes = 12

fecha_inicio = fecha_train -  timedelta(days = 30 * mes - 1)

print("Fecha Inicio",fecha_inicio)
print("Fecha Entranmiento",fecha_train)
print("Fecha Ejecucion",fecha_ejecucion)
print("Fecha Prediccion",fecha_ejecucion + timedelta(days = 7))

Fecha Inicio 2023-05-19
Fecha Entranmiento 2024-05-12
Fecha Ejecucion 2024-05-20
Fecha Prediccion 2024-05-27


In [7]:
# Spark Session
spark = create_pyspark(name="co_umv_umv", connection="GCP")

File already exists at: /jars/spark-bigquery-with-dependencies_2.12-0.26.0.jar
File already exists at: /jars/gcs-connector-hadoop3-2.2.19.jar
Process: co_umv_umv_1716490589718


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [45]:
# Data collection
df = (
    spark.read.format("bigquery")
    .option("table", f"{dataset_id}.{table_id}")
    .load()
)
# print(df.count())
df = df.drop("_PARTITIONTIME", "_PARTITIONDATE")
df.printSchema()

root
 |-- EAN: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- LOCAL_ID: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- UNITS: double (nullable = true)



In [46]:
data_transaccional = df.groupBy("LOCAL_ID", "ITEM_ID", "EAN", "DATE")\
                        .agg(F.sum("UNITS").alias("y"))\
                        .withColumnRenamed("DATE", "ds").na.drop("any")

In [18]:
data_outliers = spark.read.format("bigquery").option("table","data_analytics_default.BQ_CO_SM_SALES_03").load()

In [52]:
display(data_outliers.select(F.countDistinct("LOCAL_ID")))

,count(DISTINCT LOCAL_ID)
0,114


In [21]:
display(data_outliers.groupBy(F.col("TIPO")).count())

,TIPO,count
0,VALIDATE,671160
1,TRAIN,34516800
2,TEST,671160


In [22]:
display(data_outliers.filter(F.col("TIPO")=="VALIDATE"))

,LOCAL_ID,ITEM_ID,EAN,DIA,ds,TIPO,y,ID
0,10,0002592,7702032253135,5,2024-05-23,VALIDATE,1.0,10-0002592-0002592
1,10,0002592,7702032253135,2,2024-05-20,VALIDATE,2.0,10-0002592-0002592
2,10,0002592,7702032253135,6,2024-05-24,VALIDATE,2.0,10-0002592-0002592
3,10,0002592,7702032253135,7,2024-05-25,VALIDATE,3.0,10-0002592-0002592
4,10,0002592,7702032253135,4,2024-05-22,VALIDATE,1.0,10-0002592-0002592
5,10,0002592,7702032253135,3,2024-05-21,VALIDATE,2.0,10-0002592-0002592
6,10,0002592,7702032253135,1,2024-05-26,VALIDATE,3.0,10-0002592-0002592
7,10,0002830,7702024004677,7,2024-05-25,VALIDATE,2.0,10-0002830-0002830
8,10,0002830,7702024004677,3,2024-05-21,VALIDATE,2.0,10-0002830-0002830
9,10,0002830,7702024004677,1,2024-05-26,VALIDATE,2.0,10-0002830-0002830


In [15]:
display(data_outliers.groupBy(F.col("LOCAL_ID")).agg(F.countDistinct("ITEM_ID")).orderBy("count(ITEM_ID)"),100)

,LOCAL_ID,count(ITEM_ID)
0,203,1
1,123,2
2,204,3
3,159,3
4,11,4
...,...,...
95,27,1587
96,83,1593
97,108,1687
98,148,1706


In [54]:
display(data_outliers.groupBy(F.col("LOCAL_ID")).agg(F.countDistinct("ITEM_ID")).filter(F.col("count(ITEM_ID)")>100).orderBy("count(ITEM_ID)"),100)

,LOCAL_ID,count(ITEM_ID)
0,301,111
1,333,119
2,806,130
3,105,147
4,304,150
...,...,...
76,103,2699
77,13,2930
78,15,3912
79,14,4073


In [14]:
display(data_outliers.groupBy(F.col("LOCAL_ID")).agg(F.countDistinct("ITEM_ID")).select(F.avg("count(ITEM_ID)")))

,avg(count(ITEM_ID))
0,841.052632


In [10]:
data_outliers = data_outliers.repartition(300)

In [23]:
display(data_outliers)

,LOCAL_ID,ITEM_ID,EAN,DIA,ds,TIPO,y,ID
0,10,0002592,7702032253135,5,2023-10-12,TRAIN,1.0,10-0002592-0002592
1,10,0002592,7702032253135,5,2024-01-11,TRAIN,1.0,10-0002592-0002592
2,10,0002592,7702032253135,5,2024-02-22,TRAIN,2.0,10-0002592-0002592
3,10,0002592,7702032253135,5,2023-06-29,TRAIN,1.0,10-0002592-0002592
4,10,0002592,7702032253135,5,2023-11-30,TRAIN,1.0,10-0002592-0002592
5,10,0002592,7702032253135,5,2024-02-01,TRAIN,1.0,10-0002592-0002592
6,10,0002592,7702032253135,5,2023-06-08,TRAIN,1.0,10-0002592-0002592
7,10,0002592,7702032253135,5,2023-11-02,TRAIN,2.0,10-0002592-0002592
8,10,0002592,7702032253135,5,2023-09-21,TRAIN,1.0,10-0002592-0002592
9,10,0002592,7702032253135,5,2023-10-19,TRAIN,6.0,10-0002592-0002592


In [34]:
import time
class Model():
    def transform_input(self,dataset):
        return dataset["y"].values
    def fit(self, data):
        return []
    def predict(self, data)-> pd.DataFrame:
        return []
    def back_prediction(self,)-> pd.DataFrame:
        return []
    def __call__(self, data)-> pd.DataFrame:
        return self.predict(data)
    def apply_format(self, pronostico):
        pronostico = np.round(pronostico, 2)
        pronostico = np.where((pronostico < 0) | np.isnan(pronostico) | np.isinf(pronostico), 0, pronostico)
        return pronostico
class PD_MEDIA(Model):
    def __init__(self):
        self.name = "MEDIA"
    def fit(self, data):
        start_time = time.time()
        self.model = np.mean(self.transform_input(data))
        end_time = time.time()
        self.time = end_time - start_time
    def predict(self, data):
        return self.apply_format([self.model]*len(data))
    def back_prediction(self,)-> pd.DataFrame:
        return self.apply_format(self.model)
    
class PD_ARIMA(Model):
    def __init__(self):
        self.name = "ARIMA"
    def fit(self, data):
        start_time = time.time()
        self.model = AutoARIMA().fit(y = self.transform_input(data))
        end_time = time.time()
        self.time = end_time - start_time
    def predict(self, data):
        pronostico = self.model.predict(len(data))
        return self.apply_format(pronostico)
    def back_prediction(self,)-> pd.DataFrame:
        pronostico = self.model.predict_in_sample()
        return self.apply_format(pronostico)
    
class PD_SARIMAX(Model):
    def __init__(self):
        self.name = "SARIMAX"
    def fit(self, data):
        start_time = time.time()
        self.train_len=len(data)
        self.model = SARIMAX(endog = self.transform_input(data),
                            order = (0,0,0),
                            seasonal_order = (0,1,0,7),
                            suppress_warnings = True)\
                        .fit()
        end_time = time.time()
        self.time = end_time - start_time
    def predict(self, data):
        pronostico = self.model.forecast(len(data))
        return self.apply_format(pronostico)
    def back_prediction(self,)-> pd.DataFrame:
        pronostico = self.model.predict(start=0, end=self.train_len-1)
        return self.apply_format(pronostico)

class PD_HOLTW(Model):
    def __init__(self):
        self.name = "HOLTW"
    def fit(self, data):
        start_time = time.time()
        self.train_len=len(data)
        self.model = ExponentialSmoothing( self.transform_input(data),
                                     trend = "mul",
                                     seasonal = "mul",
                                     seasonal_periods = 7).fit()
        end_time = time.time()
        self.time = end_time - start_time
    def predict(self, data):
        pronostico = self.model.forecast(len(data))
        return self.apply_format(pronostico)
    def back_prediction(self,)-> pd.DataFrame:
        pronostico = self.model.predict(start=0, end=self.train_len-1)
        return self.apply_format(pronostico)
        
class PD_PROPHET(Model):
    def __init__(self):
        self.name = "PROPHET"
    def fit(self, data):
        start_time = time.time()
        self.train_len=len(data)
        data2 = data[["ds","y"]]
        self.model = Prophet()
        self.model.fit(data2)
        end_time = time.time()
        self.time = end_time - start_time
    def predict(self, data):
        data2 = data[["ds","y"]]
        future_pd = self.model.make_future_dataframe(
            periods=len(data2.values),
            freq='d',
            include_history = False
        )
        pronostico = self.model.predict(future_pd)
        return self.apply_format(pronostico["yhat"].values)
    def back_prediction(self,)-> pd.DataFrame:
        future_pd = self.model.make_future_dataframe(
            periods=1,
            freq='d',
            include_history = True
        )
        results_pd = self.model.predict(future_pd)
        return self.apply_format(results_pd["yhat"].iloc[:-1])
    
class PD_TREES(Model):
    def __init__(self):
        self.name = "TREES"
    def fit(self, data):
        start_time = time.time()
        self.col_names = [col for col in data.columns if col.startswith('C-')]
        X, y = data[self.col_names], data[["y"]]
        self.model = RandomForestRegressor(n_estimators=50, random_state=42)
        self.model.fit(X, y)
        self.back = self.model.predict(X)
        end_time = time.time()
        self.time = end_time - start_time
        self.most_recent_data = data.tail(1)[self.col_names]
    def predict(self, data):
#         pronostico = self.model.predict(data[self.col_names])
#         return self.apply_format(pronostico)
    
        num_days = len(data)
        most_recent_data = self.most_recent_data
        if not hasattr(self, 'model'):
            raise ValueError("The model has not been trained. Call the 'fit' method first.")
        if num_days <= 0:
            raise ValueError("Number of days to forecast must be a positive integer.")
        
        # Get the most recent data point to use as a starting point for forecasting
          # Assuming 'data' is the same data used in 'fit'
        
        forecasts = []
        for _ in range(num_days):
            forecasted_value = self.model.predict(most_recent_data)[0]
            forecasts.append(forecasted_value)
            # Update the most recent data with the forecasted value
            most_recent_data = most_recent_data.shift(-1).fillna(forecasted_value)
        
        return self.apply_format(forecasts)
    def back_prediction(self,)-> pd.DataFrame:     
        return self.apply_format(self.back)
    

class PD_MLP(Model):
    def __init__(self):
        self.name = "MLP"
        self.structure = (20, 5)
    def fit(self, data):
        start_time = time.time()
        self.col_names = [col for col in data.columns if col.startswith('C-')]
        X, y = data[self.col_names], data[["y"]]
        self.model = MLPRegressor(hidden_layer_sizes= self.structure, activation='relu', random_state=42)
        self.model.fit(X, y.values.ravel(),)  # MLPRegressor expects 1D array for y
        self.back = self.model.predict(X)
        end_time = time.time()
        self.time = end_time - start_time
        self.most_recent_data = data.tail(1)[self.col_names]
    
    def predict(self, data):
        num_days = len(data)
        most_recent_data = self.most_recent_data
        if not hasattr(self, 'model'):
            raise ValueError("The model has not been trained. Call the 'fit' method first.")
        if num_days <= 0:
            raise ValueError("Number of days to forecast must be a positive integer.")
        
        # Get the most recent data point to use as a starting point for forecasting
          # Assuming 'data' is the same data used in 'fit'
        
        forecasts = []
        for _ in range(num_days):
            forecasted_value = self.model.predict(most_recent_data)[0]
            forecasts.append(forecasted_value)
            # Update the most recent data with the forecasted value
            most_recent_data = most_recent_data.shift(-1).fillna(forecasted_value)
        
        return self.apply_format(forecasts)
    
    def back_prediction(self) -> pd.DataFrame:
        return self.apply_format(self.back)

In [35]:
def create_lagged_columns(df, column_name, num_lags):
    for i in range(1, num_lags + 1):
        df[f'C-{i}'] = df[column_name].shift(i)
    
    df = df.dropna()
    return df

models = [
    PD_HOLTW(),
    PD_PROPHET(),
    PD_SARIMAX(),
    PD_MEDIA(),
    PD_ARIMA(),
#     PD_TREES(),
#     PD_MLP(),
#     PD_MLP2(),
#     PD_MLP3(),
#     PD_MLP4(),
#     PD_MLP5(),
]
def find_key_with_min_value(my_dict):
    """
    Find the key with the minimum value in a dictionary.
    
    Args:
        my_dict (dict): The input dictionary.
    
    Returns:
        The key with the minimum value.
    """
    min_key = None
    min_value = float('inf')
    
    for key, value in my_dict.items():
        if value < min_value:
            min_key = key
            min_value = value
    
    return min_key
def forecast_generator(models):

    def fn_forecast(data : pd.DataFrame) -> pd.DataFrame:
        data = data.sort_values("ds")
#         data = create_lagged_columns(data, "y", 5)
        data_train = data.query('TIPO == "TRAIN"')\
                         .sort_values("ds")
        data_test = data.query('TIPO == "TEST"')\
                        .sort_values("ds")\
                        .reset_index()
        
        data_forecast = data_test[["LOCAL_ID", "ITEM_ID", "EAN", "ds", "y"]]
        
        mape_scores = {}
        rmse_scores = {}
        flag_mape = False
        mape_limit = COTA_MAPE_MODELS
        n_models_ensemble = ENSEMBLE
        
        for index, model in enumerate(models):
            if(flag_mape and index>=n_models_ensemble):
                data_forecast[f"F_{model.name}"] = data_forecast[f"F_{models[0].name}"]
                mape_scores[model.name] = mape * 1.1
                rmse_scores[model.name] = rmse * 1.1
            model.fit(data_train)
            predictions = model.predict(data_test).reshape(-1)
#             print(predictions, data_test['y'])
            
            data_forecast[f"F_{model.name}"] = predictions
    
            # Calculate MAPE
            absolute_errors = np.abs(data_test['y'] - predictions)

            percentage_errors = (absolute_errors / data_test['y']) * 100
            mape = np.mean(percentage_errors)
            mape_scores[model.name] = mape
            
            # Calculate RMSE
            try:
                rmse = sqrt(mean_squared_error(data_test['y'], predictions))
            except ValueError:
                rmse = float('inf')
            rmse_scores[model.name] = rmse
            
            if mape < mape_limit:
                flag_mape = True
        
        tmp = data_forecast[[f"F_{m.name}" for m in models[:n_models_ensemble]]].loc[:, (data_forecast != 0).any(axis=0)].mean(axis=1)
        tmp = np.round(tmp, 2)
        data_forecast["F_ENSEMBLE"] = np.where((tmp < 0) | (np.isnan(tmp)), 0, tmp)
        
        # Calculate MAPE and RMSE for the ensemble model
        absolute_errors_ensemble = np.abs(data_test['y'] - data_forecast["F_ENSEMBLE"])
        percentage_errors_ensemble = (absolute_errors_ensemble / data_test['y']) * 100
        mape_ensemble = np.mean(percentage_errors_ensemble)
        try:
            rmse_ensemble = sqrt(mean_squared_error(data_test['y'], data_forecast["F_ENSEMBLE"]))
        except ValueError:
            rmse_ensemble =  float('inf')
        mape_scores["ENSEMBLE"] = mape_ensemble
        rmse_scores["ENSEMBLE"] = rmse_ensemble
                
        # Find the best model according to RMSE and MAPE
        best_model_mape = find_key_with_min_value(mape_scores)
        best_model_rmse = find_key_with_min_value(rmse_scores)
        
        data_forecast["RMSE"] = best_model_rmse
        data_forecast["MAPE"] = best_model_mape
        data_forecast["RMSE_VALUE"] = rmse_scores[best_model_rmse]
        data_forecast["MAPE_VALUE"] = mape_scores[best_model_mape]
                               
        return data_forecast
    
    return fn_forecast

In [36]:
estructura_forecast = StructType([
    StructField("LOCAL_ID", StringType()),
    StructField("ITEM_ID", StringType()),
    StructField("EAN", StringType()),
    StructField("ds", DateType()),
    StructField("y", DoubleType()),
    *[StructField(f"F_{m.name}", DoubleType()) for m in models],
    StructField("F_ENSEMBLE", DoubleType()),
    StructField("RMSE", StringType()),
    StructField("MAPE", StringType()),
    StructField("RMSE_VALUE", DoubleType()),
    StructField("MAPE_VALUE", DoubleType()),
])

# Se obtiene la data mape 
data_forecast = data_outliers\
    .groupBy("LOCAL_ID", "ITEM_ID", "EAN")\
    .applyInPandas(forecast_generator(models), schema = estructura_forecast )\
    .withColumn("FECHA_EJECUCION", F.lit(fecha_ejecucion))

In [37]:
# if MONDAY:
#     data_forecast.write.mode("overwrite").parquet(f"gs://{ENV}-bucket-dataproc-bigquery/dataproc/umv/tmp/co/sm/data_forecast")
data_forecast = spark.read.parquet(f"gs://{ENV}-bucket-dataproc-bigquery/dataproc/umv/tmp/co/sm/data_forecast")
display(data_forecast)

,LOCAL_ID,ITEM_ID,EAN,ds,y,F_HOLTW,F_PROPHET,F_SARIMAX,F_MEDIA,F_ARIMA,F_ENSEMBLE,RMSE,MAPE,RMSE_VALUE,MAPE_VALUE,FECHA_EJECUCION
0,10,0013996,7702506112197,2024-05-13,2.0,1.95,1.88,1.0,1.94,2.61,1.92,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
1,10,0013996,7702506112197,2024-05-14,1.0,1.97,1.88,2.0,1.94,2.27,1.92,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
2,10,0013996,7702506112197,2024-05-15,1.0,1.23,1.10,1.0,1.94,1.98,1.16,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
3,10,0013996,7702506112197,2024-05-16,1.0,1.25,1.16,1.0,1.94,1.72,1.20,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
4,10,0013996,7702506112197,2024-05-17,2.0,1.96,1.90,2.0,1.94,1.50,1.93,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
5,10,0013996,7702506112197,2024-05-18,1.0,2.11,2.03,3.0,1.94,1.30,2.07,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
6,10,0013996,7702506112197,2024-05-19,4.0,2.85,2.78,3.0,1.94,1.13,2.82,PROPHET,PROPHET,0.695259,36.928571,2024-05-20
7,10,0027749,7703812101356,2024-05-13,10.0,3.41,3.20,4.0,4.94,2.59,3.30,PROPHET,PROPHET,3.195117,32.011905,2024-05-20
8,10,0027749,7703812101356,2024-05-14,2.0,2.79,2.45,2.0,4.94,4.20,2.62,PROPHET,PROPHET,3.195117,32.011905,2024-05-20
9,10,0027749,7703812101356,2024-05-15,3.0,2.96,2.61,1.0,4.94,3.38,2.78,PROPHET,PROPHET,3.195117,32.011905,2024-05-20


In [38]:
data_mape = data_forecast.groupBy("LOCAL_ID", "ITEM_ID", "EAN").agg(F.first("MAPE").alias("MODELO"),
                                                                    F.first("MAPE_VALUE").alias("MAPE"),
                                                                    F.first("RMSE_VALUE").alias("RMSE"))

In [39]:
display(data_mape)

,LOCAL_ID,ITEM_ID,EAN,MODELO,MAPE,RMSE
0,10,0000633,482,SARIMAX,67.924150,0.562374
1,10,0007061,7702085012246,SARIMAX,57.142857,0.830258
2,10,0007172,7702085013076,PROPHET,78.265201,4.097834
3,10,0007934,7702047003497,ARIMA,36.142857,0.363613
4,10,0008430,7702231000035,SARIMAX,42.857143,1.448684
5,10,0010047,7702035432117,SARIMAX,28.571429,1.281210
6,10,0010056,7702035432339,ENSEMBLE,51.611284,20.638907
7,10,0013996,7702506112197,PROPHET,36.928571,0.695259
8,10,0016841,7702432002005,ENSEMBLE,20.360806,2.784536
9,10,0027744,7703812101103,HOLTW,34.017007,1.245695


In [40]:
average_MAPE = data_mape.select(F.avg("MAPE")).collect()[0][0]
average_RMSE = data_mape.select(F.avg("RMSE")).collect()[0][0]

In [41]:
print(average_MAPE,average_RMSE)

41.78130924809774 2.346254104550151


In [42]:
data_modelos = data_outliers\
                    .join(data_mape,
                          how = "left",
                          on = ["LOCAL_ID", "ITEM_ID", "EAN"])

In [43]:
def umv_generator(models):

    def fn_forecast(data : pd.DataFrame) -> pd.DataFrame:
        data = data.sort_values("ds")
        data = create_lagged_columns(data, "y", 10)
        
        data_train = data.query('TIPO != "VALIDATE"')\
                         .sort_values("ds")
        data_test = data.query('TIPO == "VALIDATE"')\
                        .sort_values("ds")\
                        .reset_index()

        model_selected = data["MODELO"].iloc[0]
        n_models_ensemble = ENSEMBLE
        data_forecast = data_test[["LOCAL_ID", "ITEM_ID", "EAN", "ds", "MODELO", "MAPE"]]

        
        model = None
        for m in models:
            if m.name == model_selected:
                model = m
                break
        if model:
            model.fit(data_train)
            data_forecast[f"VALIDATE"] = model.predict(data_test)
        else:
            for m in models[:n_models_ensemble]:
                m.fit(data_train)
                data_forecast[f"F_{m.name}"] = m.predict(data_test)

            tmp = data_forecast[[f"F_{m.name}" for m in models[:n_models_ensemble]]].loc[:, (data_forecast != 0).any(axis=0)].mean(axis=1)
            tmp = np.round(tmp, 2)
            data_forecast["VALIDATE"] = np.where( (tmp < 0) | (np.isnan(tmp)), 0, tmp)
            data_forecast = data_forecast.drop(columns=[f"F_{m.name}" for m in models[:n_models_ensemble]])
        return data_forecast
    return fn_forecast

In [47]:
# Se define la cota del mape
cota_mape = 0.35

# Se define la significancia
alpha = 0.10
z_alpha = np.round(sps.norm.ppf(1-(alpha/2)), 2)

# Se definen los cortes para la data final
cota_inferior_pronostico = 4
cota_superior_mape = 31
cota_inferior_mape = 4

# Se define la estructura de la data mape
estructura_umv = StructType([
    StructField("LOCAL_ID", StringType()),
    StructField("ITEM_ID", StringType()),
    StructField("EAN", StringType()),
    StructField("ds", DateType()),
    StructField("MODELO", StringType()),
    StructField("MAPE", DoubleType()),
    StructField("VALIDATE", DoubleType())
])

# Se obtiene la data mape 
data_output = data_modelos\
               .groupBy("LOCAL_ID", "ITEM_ID", "EAN")\
               .applyInPandas(umv_generator(models), schema = estructura_umv)\
               .withColumn("UMV", 
                           F.when(F.col("MAPE")/100 <= F.lit(cota_mape), F.round(F.col("VALIDATE")*(1 - 2*F.col("MAPE")/100),2))\
                            .otherwise(F.round(F.col("VALIDATE") - (F.lit(z_alpha)*F.col("MAPE")/100)/sqrt(7),2)))\
               .withColumn("UMV",
                           F.when(F.col("UMV") <= 1, 1)\
                            .otherwise(F.col("UMV")))\
               .withColumn("DIA_SEMANA", F.dayofweek(F.col("ds")))\
               .withColumn("DIA",
                           F.when(F.col("DIA_SEMANA") == 2, "LUNES")\
                            .when(F.col("DIA_SEMANA") == 3, "MARTES")\
                            .when(F.col("DIA_SEMANA") == 4, "MIERCOLES")\
                            .when(F.col("DIA_SEMANA") == 5, "JUEVES")\
                            .when(F.col("DIA_SEMANA") == 6, "VIERNES")\
                            .when(F.col("DIA_SEMANA") == 7, "SABADO")\
                            .when(F.col("DIA_SEMANA") == 1, "DOMINGO"))\
               .select("LOCAL_ID", "ITEM_ID", "EAN", "ds", "DIA", "DIA_SEMANA",
                       "VALIDATE", "UMV", "MAPE", "MODELO")\
               .withColumn("UMV", F.floor(F.col("UMV")))\
               .withColumn("MAPE", F.floor(F.col("MAPE")))\
               .withColumn("VALIDATE", F.floor(F.col("VALIDATE")))\
               .filter((F.col("VALIDATE") > F.lit(cota_inferior_pronostico)) &
                       (F.col("MAPE") < F.lit(cota_superior_mape)) & 
                       (F.col("MAPE") > F.lit(cota_inferior_mape)))\
               .join(data_transaccional\
                         .select("LOCAL_ID", "ITEM_ID", "EAN")\
                         .distinct()\
                         .withColumnRenamed("ds", "Fecha"),
                     on = ["LOCAL_ID", "ITEM_ID", "EAN"],
                     how = "left")\
                .withColumnRenamed("ds", "DATE")\
                .select("LOCAL_ID", "ITEM_ID", "EAN", "DATE", "DIA", "DIA_SEMANA",
                        "VALIDATE", "UMV", "MAPE", "MODELO").withColumnRenamed("VALIDATE","FORECAST")\
                .withColumnRenamed("DIA","DAY")\
                .withColumnRenamed("DIA_SEMANA","WEEK_DAY")\
                .withColumnRenamed("MODELO","MODEL")


In [ ]:
# if MONDAY:
#     data_output.write.mode("overwrite").parquet("/tmp/co/sm/data_output")
#     data_output = spark.read.parquet("/tmp/co/sm/data_output")
#     tmpd = data_output.groupBy("MODEL").count().toPandas()
#     tmpd.sort_values("count")

In [ ]:
# if MONDAY:
data_output.write.parquet(f"gs://{ENV}-bucket-dataproc-bigquery/dataproc/umv/tmp/co/sm/data_output_umv")

In [ ]:
data_output = spark.read.parquet(f"gs://{ENV}-bucket-dataproc-bigquery/dataproc/umv/tmp/co/sm/data_output_um")


AnalysisException: Path does not exist: gs://staging-bucket-dataproc-bigquery/dataproc/umv/tmp/co/sm/data_output_um

In [ ]:
print("DONE")

In [8]:
# if MONDAY:
data_output = spark.read.format("bigquery").option("table", "data_analytics_default.BQ_CO_SM_UMV_01").load()
display(data_output.groupBy("DATE").count().orderBy(F.col("DATE").desc()))    

,DATE,count
0,2024-05-26,5376
1,2024-05-25,6002
2,2024-05-24,4138
3,2024-05-23,3872
4,2024-05-22,4006
5,2024-05-21,4137
6,2024-05-20,4601
7,2024-04-07,5743
8,2024-04-06,5794
9,2024-04-05,4493


In [12]:
    data_mape = spark.read.parquet("/tmp/co/sm/data_mape")
    display(data_mape)

,LOCAL_ID,ITEM_ID,EAN,MODELO,MAPE
0,10,0013996,7702506112197,PROPHET,27.880952
1,10,0027749,7703812101356,HOLTW,28.842857
2,10,0851911,7702001106660,SARIMAX,14.285714
3,10,0963186,7702766098057,PROPHET,27.785714
4,10,3251527,7707237414091,SARIMAX,78.571429
5,10,3328671,7703616274645,HOLTW,49.071429
6,103,0005010,7702137630206,HOLTW,33.130952
7,103,0007466,7702001044122,ENSEMBLE,26.407143
8,103,0099670,7702001121748,PROPHET,51.145238
9,103,0245696,7702026062163,MEDIA,105.726984


In [14]:
average_score = data_mape.select(F.avg("MAPE")).collect()[0][0]
average_score

40.98317382053177